In [1]:
%run 'Load_base_data.ipynb'

In [2]:
fire_incidents = pd.read_csv('RawData/Fire_Incidents.csv', low_memory=False)

In [3]:
fire_incidents.head()

,Incident Number,Exposure Number,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,Zipcode,...,Detector Effectiveness,Detector Failure Reason,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,Neighborhood District,Location
0,9030109,0,310 Colon Av.,04/12/2009,91020273,04/12/2009 06:09:13 PM,04/12/2009 06:13:45 PM,04/12/2009 07:23:13 PM,SF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13067402,0,20 Lansdale Av,07/18/2013,131990117,07/18/2013 10:32:03 AM,07/18/2013 10:37:15 AM,07/18/2013 10:39:55 AM,SF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12044490,0,7th St. / Folsom St.,05/13/2012,121340051,05/13/2012 03:55:37 AM,05/13/2012 04:01:57 AM,05/13/2012 04:05:44 AM,SF,94103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,South of Market,"(37.7767460000297, -122.407844)"
3,13033326,0,2799 Pacific Av,04/09/2013,130990286,04/09/2013 04:34:07 PM,04/09/2013 04:39:31 PM,04/09/2013 05:20:27 PM,SF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11101416,0,Polk St. / Pine St.,11/01/2011,113050357,11/01/2011 06:07:45 PM,11/01/2011 06:10:17 PM,11/01/2011 06:11:09 PM,SF,94109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Nob Hill,"(37.7896190000297, -122.420497)"


In [5]:
fire_incidents['Fire Code'] =fire_incidents['Primary Situation'].str.extract('(\d+)', expand=False)

In [6]:
#subset the data so that only buildings on fire are in this dataset-- that the positive class for the classifiction model
fire_incidents_building_fires = fire_incidents[(fire_incidents['Fire Code']=='111')]

In [7]:
fire_incidents_building_fires.shape

(6467, 64)

In [8]:
# get Lat/Long
fire_incidents_building_fires['Lat']=fire_incidents_building_fires['Location'].apply(get_latitude)
fire_incidents_building_fires['Long']=fire_incidents_building_fires['Location'].apply(get_longitude)
fire_incidents_building_fires['BuildingFire'] =1 # response variable 

/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [9]:
# convert to geopandas df
fire_incidents_building_fires['Coordinates'] = list(zip(fire_incidents_building_fires.Long, 
                                                      fire_incidents_building_fires.Lat))
fire_incidents_building_fires['Coordinates'] = fire_incidents_building_fires['Coordinates'].apply(Point)

/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [11]:
#remove datapoints without Lat/Long
fire_incidents_main=fire_incidents_building_fires.dropna(subset=['Lat','Long'])

In [12]:
fire_incidents_main.shape

(5110, 68)

In [13]:
fire_incidents_main_gdf = gpd.GeoDataFrame(fire_incidents_main, geometry='Coordinates')

In [14]:
#spatial join gps 
fire_incidents_main_parcels = gpd.sjoin(fire_incidents_main_gdf, geo_loc_gdf_sub, how="left", op='intersects')

In [16]:
fire_incidents_w_parcel = pd.DataFrame(fire_incidents_main_parcels)
#fire_incidents_w_parcel.drop('Coordinates', inplace= True)

In [21]:
fire_incidents_w_parcel.shape

(10546, 70)

In [23]:
#count the total number of fires per parcel
fire_incidents_per_building =fire_incidents_w_parcel[['BuildingFire','blklot']].groupby(['blklot']).agg('count')
fire_incidents_per_building.reset_index(inplace=True)
fire_incidents_per_building.head()
fire_incidents_per_building.shape

(6963, 2)

In [25]:
R2Parcel = pd.read_csv('residential_units.csv')

In [29]:
#only get r2 data 
fire_incidents_clean = R2Parcel.merge(fire_incidents_per_building,left_on=['Block and Lot Number'], 
                                                      right_on=['blklot'], how='inner')

In [30]:
#check for duplicates
print(fire_incidents_clean.shape)
print(fire_incidents_clean.drop_duplicates('blklot').shape)

(617, 6)
(617, 6)


In [31]:
#save to csv
fire_incidents_clean.to_csv('fire_incidents_clean.csv')

In [32]:
fire_incidents_clean.blklot.isnull().sum()

0

### Fires per year for data viz.

In [35]:
#calculate total fires per year
fire_incidents_building_fires['Incident Date']=pd.to_datetime(fire_incidents_building_fires['Incident Date'],format='%m/%d/%Y')
fire_incidents_building_fires['Year']=fire_incidents_building_fires['Incident Date'].apply(lambda x: x.year)
fire_incidents_building_fires['Num_fires']=1
fireloss =fire_incidents_building_fires[['Year','Estimated Property Loss',
                              'Fire Fatalities', 'Fire Injuries',
                              'Civilian Fatalities', 'Civilian Injuries',
                                        'Num_fires']].groupby('Year').agg(sum)
fireloss.reset_index(inplace=True)
fireloss.to_csv('fires_per_year.csv')

/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/taniajogesh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste